**MOUNT GOOGLE DRIVE**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
cd drive

[Errno 2] No such file or directory: 'drive'
/content/drive/My Drive


In [ ]:
cd My Drive

/content/drive/My Drive


**EXTRACTING** **DATA**

In [10]:
import pandas as pd
d=pd.read_csv('/content/drive/MyDrive/Customer_Churn.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Customer_Churn.csv'

# New Section

In [ ]:
d.sample(5)


In [ ]:
d.drop('customerID',axis='columns',inplace=True)

In [ ]:
d['Churn'].value_counts()

In [ ]:
d.sample(15)

In [ ]:
d.dtypes

In [ ]:
d.TotalCharges.values

In [ ]:
pd.to_numeric(d.TotalCharges,errors='coerce').isnull()

In [ ]:
d.TotalCharges = pd.to_numeric(d.TotalCharges, errors='coerce')
d.isnull().sum()

In [ ]:
d[pd.to_numeric(d.TotalCharges,errors='coerce').isnull()]

Removing missing value

In [ ]:
#Removing missing values
d.dropna(how = 'any', inplace = True)


In [ ]:
d1 = d[d.TotalCharges!=' ']
d1.shape

In [ ]:
d1.dtypes

In [ ]:
d1.TotalCharges = pd.to_numeric(d1.TotalCharges)

In [ ]:
d1.TotalCharges.values

In [ ]:
d1[d1.Churn=='No']

In [ ]:
def print_unique_col_values(d):
       for column in d:
            if d[column].dtypes=='object':
                print(f'{column}: {d[column].unique()}')

In [ ]:
print_unique_col_values(d1)

In [ ]:
d1.replace('No internet service','No',inplace=True)
d1.replace('No phone service','No',inplace=True)

In [ ]:
print_unique_col_values(d1)

In [ ]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
for col in yes_no_columns:
    d1[col].replace({'Yes': 1,'No': 0},inplace=True)

In [ ]:
for col in d1:
    print(f'{col}: {d1[col].unique()}')

In [ ]:
d1['gender'].replace({'Female':1,'Male':0},inplace=True)

In [ ]:
d1.gender.unique()

In [ ]:
d2 = pd.get_dummies(data=d1, columns=['InternetService','Contract','PaymentMethod'])
d2.columns

In [ ]:
d2.sample(5)

In [ ]:
d2.dtypes

In [ ]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
d2[cols_to_scale] = scaler.fit_transform(d2[cols_to_scale])

In [ ]:
for col in d2:
    print(f'{col}: {d2[col].unique()}')

In [ ]:
d2.sample(10)

In [ ]:
X = d2.drop('Churn',axis='columns')
y = d2['Churn']


**SPLITTING THE DATA**


In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)


In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train[:10]


In [ ]:
y_train

In [ ]:
len(X_train.columns)

**CNN MODEL BUILDING**


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Convert labels to one-hot encoded vectors
num_classes = 2
y_train_one_hot = to_categorical(y_train, num_classes)
y_val_one_hot = to_categorical(y_test, num_classes)

# Convert input features to NumPy arrays and reshape
X_train_array = X_train.to_numpy().reshape((X_train.shape[0], 26, 1, 1))
X_val_array = X_test.to_numpy().reshape((X_test.shape[0], 26, 1, 1))



# Define the model
model = keras.Sequential([
    keras.layers.Reshape((26, 1, 1), input_shape=(26,)),
    keras.layers.Conv2D(32, (3, 1), activation='relu'),
    keras.layers.MaxPooling2D((2, 1)),
    keras.layers.Conv2D(64, (3, 1), activation='relu'),
    keras.layers.MaxPooling2D((2, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(num_classes, activation='softmax')
])
# Compile the model
model.compile(
              loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
# Train the model with validation data
epochs = 30
batch_size = 32
history = model.fit(X_train_array, y_train_one_hot,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val_array, y_val_one_hot))

In [ ]:
model.evaluate(X_val_array, y_val_one_hot)

In [ ]:
yp = model.predict(X_val_array)



In [ ]:
model.save("model_weights.h5")

In [ ]:
import matplotlib.pyplot as plt

# Assuming X_test, y_test are your test data
# Convert labels to one-hot encoded vectors for the test set
y_test_one_hot = to_categorical(y_test, num_classes)

# Convert input features to NumPy arrays and reshape for the test set
X_test_array = X_test.to_numpy().reshape((X_test.shape[0], 26, 1, 1))

# Predict on the test set
y_pred_one_hot = model.predict(X_test_array)

# Convert one-hot encoded predictions and true labels back to class labels
y_pred_classes = np.argmax(y_pred_one_hot, axis=1)
y_true_classes = np.argmax(y_test_one_hot, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

# Visualize the confusion matrix using seaborn
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f'class{i}' for i in range(num_classes)],
            yticklabels=[f'class{i}' for i in range(num_classes)])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# Predict on the test set
y_pred_one_hot = model.predict(X_test_array)

# Convert one-hot encoded predictions and true labels back to class labels
y_pred_classes = np.argmax(y_pred_one_hot, axis=1)
y_true_classes = np.argmax(y_test_one_hot, axis=1)

# Calculate metrics
precision, recall, f1, _ = precision_recall_fscore_support(y_true_classes, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_true_classes, y_pred_classes)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')

In [ ]:
# Plot epoch vs accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy vs Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()



In [ ]:
# Plot epoch vs loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss vs Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
!pip install -U gradio

In [ ]:
!pip install gradio

**IMPLEMENTATION**

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load model
model = tf.keras.models.load_model('model_weights.h5')



# Load label encoders for categorical features
label_encoders = {}  # Dictionary to store label encoders

# Function to preprocess input features
def preprocess_input(input_data):
    # Encode categorical features
    for feature, encoder in label_encoders.items():
        input_data[feature] = encoder.transform([input_data[feature]])[0]

    # Convert input data to a numpy array
    input_array = np.array(list(input_data.values())).reshape(1, -1)

    return input_array

def predict_churn(gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService,
                  MultipleLines,InternetService_DSL,InternetService_Fiberoptic,InternetService_No,OnlineSecurity,OnlineBackup,DeviceProtection,
                  TechSupport,StreamingTV,StreamingMovies,Contract_Month_to_month,Contract_Oneyear,Contract_Twoyear,PaperlessBilling,
                  PaymentMethod_Banktransfer,PaymentMethod_Creditcard,PaymentMethod_Electroniccheck,PaymentMethod_Mailedcheck,MonthlyCharges,
                  TotalCharges
                 ):


    # Map radio button values to integers
    radio_mapping = {"1": 1, "0": 0}

    # Convert radio button inputs to numerical values

    input_data = np.array([
    radio_mapping[gender], radio_mapping[SeniorCitizen], radio_mapping[Partner],
    radio_mapping[Dependents], tenure, radio_mapping[PhoneService],
    radio_mapping[MultipleLines], radio_mapping[InternetService_DSL], radio_mapping[InternetService_Fiberoptic],
    radio_mapping[InternetService_No], radio_mapping[OnlineSecurity],
    radio_mapping[OnlineBackup], radio_mapping[DeviceProtection],
    radio_mapping[TechSupport],radio_mapping[StreamingTV],radio_mapping[StreamingMovies], radio_mapping[Contract_Month_to_month],
    radio_mapping[Contract_Oneyear], radio_mapping[Contract_Twoyear],
    radio_mapping[PaperlessBilling],
    radio_mapping[PaymentMethod_Banktransfer], radio_mapping[PaymentMethod_Creditcard],
    radio_mapping[PaymentMethod_Electroniccheck], radio_mapping[PaymentMethod_Mailedcheck],MonthlyCharges,TotalCharges,
   ])


    input_array = input_data.reshape(1,-1)

    pred = model.predict(input_array)[0][0]

    return "churn" if pred==1 else "Not churn"




# Define input and output interfaces
input_interface = gr.Interface(
    fn=predict_churn,
    inputs=[
        gr.Radio(["1", "0"], label="Gender"),
        gr.Radio(["1","0"],label="SeniorCitizen"),
        gr.Radio(["1","0"],label="Partner"),
        gr.Radio(["1","0"],label="Dependents"),
        gr.Number(label="tenure"),
        gr.Radio(["1","0"], label="PhoneService"),
        gr.Radio(["1","0"], label="MultipleLines"),
        gr.Radio(["1","0"], label="InternetService_DSL"),
        gr.Radio(["1","0"], label="InternetService_Fibre_optics"),
        gr.Radio(["1","0"], label="InternetService_No"),
        gr.Radio(["1","0"], label="OnlineSecurity"),
        gr.Radio(["1","0"], label="OnlineBackup"),
        gr.Radio(["1","0"], label="DeviceProtection"),
        gr.Radio(["1","0"], label="TechSupport"),
        gr.Radio(["1","0"], label="StreamingTV"),
        gr.Radio(["1","0"], label="StreamingMovies"),
        gr.Radio(["1","0"], label="Contract_Month-to-month"),
        gr.Radio(["1","0"], label="Contract_oneYear"),
        gr.Radio(["1","0"], label="Contract_TwoYear"),
        gr.Radio(["1","0"], label="PaperlessBilling"),
        gr.Radio(["1","0"], label="PaymentMethod_Banktransfer(automatic)"),
        gr.Radio(["1","0"], label="PaymentMethod_Credit card (automatic)"),
        gr.Radio(["1","0"], label="PaymentMethod_Electroniccheck"),
        gr.Radio(["1","0"], label="PaymentMethod_Mailedcheck"),
        gr.Number(["1","0"], label="MonthlyCharges"),
        gr.Number(["1","0"], label="TotalCharges"),
    ],
    outputs=gr.Label()
)

# Launch the app
input_interface.launch(debug=True)
